### from PROCEDURE dpdw.dbo.SP_T_DIM_FND_COM_COUNTRY_N_C

In [0]:
# %run "../include/functions"

In [0]:
%sql
TRUNCATE TABLE dpdw.dbo.T_DIM_FND_COM_COUNTRY2;

In [0]:
%sql
USE CATALOG fed_syn_catalog;

In [0]:
query = f"""
INSERT INTO dpdw.dbo.T_DIM_FND_COM_COUNTRY2 -- FIXME dpdw.dbo.T_DIM_FND_COM_COUNTRY  -- INSERT INTO [dbo].[T_DIM_FND_COM_COUNTRY]
    (
       COUNTRY_CODE           --국가코드  -- 	   [COUNTRY_CODE]                                          --국가코드
      ,COUNTRY_NAME           --국가명  -- 	  ,[COUNTRY_NAME]                                          --국가명
      ,COUNTRY_CODE_DESC      --국가코드설명  -- 	  ,[COUNTRY_CODE_DESC]                                     --국가코드설명
      ,TERRITORY_ID           --권역ID  -- 	  ,[TERRITORY_ID]                                          --권역ID
      ,TERRITORY_ENGLISH_NAME --권역영문명  -- 	  ,[TERRITORY_ENGLISH_NAME]                                --권역영문명
      ,DOMESTIC_OVERSEAS_TYPE --국내해외구분코드  -- 	  ,[DOMESTIC_OVERSEAS_TYPE]                                --국내해외구분코드
      ,USAGE_FLAG             --사용여부  -- 	  ,[USAGE_FLAG]                                            --사용여부
      ,MERGED_NAME            --권역국가명  -- 	  ,[MERGED_NAME]                                           --권역국가명
      ,ETL_DT                 --적재일시  -- 	  ,[ETL_DT]                                                --적재일시
    )
    SELECT IFNULL(FTS.TERRITORY_CODE,'z{')                        AS COUNTRY_CODE -- SELECT ISNULL(FTS.TERRITORY_CODE,'z{')                        AS COUNTRY_CODE
            ,IFNULL(FTT.TERRITORY_SHORT_NAME, '데이타 없음')          AS COUNTRY_NAME  -- ,ISNULL(FTT.TERRITORY_SHORT_NAME, N'데이타 없음')          AS COUNTRY_NAME
            ,FTT.DESCRIPTION                                        AS COUNTRY_CODE_DESC
            ,IFNULL(RT.TERRITORY_ID,-99)                            AS TERRITORY_ID  -- ,ISNULL(RT.TERRITORY_ID,-99)                            AS TERRITORY_ID
            ,RT.SEGMENT1                                            AS TERRITORY_ENGLISH_NAME
            ,CASE WHEN TERRITORY_ID = 1211 THEN 'D'
                  ELSE 'E'
              END                                                    AS DOMESTIC_OVERSEAS_TYPE
            ,RT.ENABLED_FLAG                                        AS USAGE_FLAG
            ,RT.DESCRIPTION                                         AS MERGED_NAME --EC.MERGED_NAME
            ,DATEADD(HOUR, 9 ,GETDATE())                            AS ETL_DT
        FROM ERPSYS.ERP_FND_TERRITORIES_TL  FTT
        JOIN ERPSYS.ERP_FND_TERRITORIES   FTS
          ON FTT.TERRITORY_CODE = FTS.TERRITORY_CODE
        LEFT OUTER
        JOIN ERPSYS.ERP_RA_TERRITORIES RT
          ON FTT.TERRITORY_CODE = RT.SEGMENT2
    AND RT.STATUS = 'A' -- ERP 기준 유효한 것 체크, 코드가 중복됨.
        UNION ALL
      SELECT CASE WHEN T.SEQNUM = 1 THEN 'z{'
                  ELSE 'z~'
              END                                                    AS COUNTRY_CODE
            ,CASE WHEN T.SEQNUM = 1 THEN '데이터 없음'  -- ,CASE WHEN T.SEQNUM = 1 THEN N'데이터 없음'
                  ELSE '데이터 오류'  -- ELSE N'데이터 오류'
              END                                                    AS COUNTRY_NAME
            ,CASE WHEN T.SEQNUM = 1 THEN '데이터 없음' -- ,CASE WHEN T.SEQNUM = 1 THEN N'데이터 없음'
                  ELSE '데이터 오류'  -- ELSE N'데이터 오류'
              END                                                    AS COUNTRY_CODE_DESC
            ,CASE WHEN T.SEQNUM = 1 THEN -99
                  ELSE -999
              END                                                    AS TERRITORY_ID
            ,NULL                                                   AS TERRITORY_ENGLISH_NAME
            ,NULL                                                   AS DOMESTIC_OVERSEAS_TYPE
            ,'Y'                                                    AS USAGE_FLAG
            ,NULL                                                   AS MERGED_NAME
            ,DATEADD(HOUR, 9 ,GETDATE())                            AS ETL_DT
        FROM (
              SELECT ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS SEQNUM	-- SELECT ROW_NUMBER() OVER (ORDER BY (SELECT NULL))     AS SEQNUM
                FROM (
                      SELECT 1 AS NN
                        FROM INFORMATION_SCHEMA.COLUMNS A1
                      ) A
              ) T
        WHERE SEQNUM <= 2
      ;
"""

In [0]:
num_rows_inserted = 0
try:
    sqlresult = spark.sql(query)
    num_rows_inserted = sqlresult.collect()[0]['num_inserted_rows']
except Exception as e:
    err_msg = str(e)
    error_code = type(e).__name__
finally:
    insert_log("SP_T_DIM_FND_COM_COUNTRY_N_C", num_rows_inserted, err_msg, error_code)
    # CALL dpdw.dbo.SP_ETL_DATA_INSERT_LOG(v_run_pgm, v_tgt_job_area, v_parm_from, v_parm_to, v_st_date, v_load_cnt, v_err_mesg, v_pgm_status);